In [ ]:
import cv2
import dlib
import playsound
from threading import Thread
from scipy.spatial import distance
cap = cv2.VideoCapture(0)
hog_face_detector = dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")
tracker = cv2.TrackerKCF_create()
dlib_facelandmark = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def sound_alarm(path):
	playsound.playsound(path)
def calculate_EAR(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear_aspect_ratio = (A+B)/(2.0*C)
	return ear_aspect_ratio
def calculate_mouth(m):
	A = distance.euclidean(m[2], m[10])
	B = distance.euclidean(m[4], m[8])
	C = distance.euclidean(m[0], m[6])
	ear_aspect_ratio = (A+B)/(2.0*C)
	return ear_aspect_ratio


while True:
    _, frame = cap.read()
    cv2.putText(frame,"Detecting face",(20,100),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),4)
    cv2.imshow("Detecting face",frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = hog_face_detector(gray)
    for face in faces:
        x = face.rect.left()
        y = face.rect.top()
        w = face.rect.right() - x
        h = face.rect.bottom() - y
        bbox = (x,y,w,h)
        ok = tracker.init(frame, bbox)
        cv2.destroyWindow("Detecting face")
        alarm=False
        e=0
        m=0
        while True:
            ok, frame = cap.read()
            ok, bbox = tracker.update(frame)

            if ok:
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]),int(bbox[1] + bbox[3]))
                g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                b=dlib.rectangle(int(bbox[0]), int(bbox[1]), int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                face_landmarks = dlib_facelandmark(g, b)
                leftEye = []
                rightEye = []
                mouth=[]

                for n in range(36,42):
                    x = face_landmarks.part(n).x
                    y = face_landmarks.part(n).y
                    leftEye.append((x,y))
                    next_point = n+1
                    if n == 41:
                        next_point = 36
                    x2 = face_landmarks.part(next_point).x
                    y2 = face_landmarks.part(next_point).y
                    cv2.line(frame,(x,y),(x2,y2),(0,255,0),1)

                for n in range(42,48):
                    x = face_landmarks.part(n).x
                    y = face_landmarks.part(n).y
                    rightEye.append((x,y))
                    next_point = n+1
                    if n == 47:
                        next_point = 42
                    x2 = face_landmarks.part(next_point).x
                    y2 = face_landmarks.part(next_point).y
                    cv2.line(frame,(x,y),(x2,y2),(0,255,0),1)
                for n in range(48,61):
                    x = face_landmarks.part(n).x
                    y = face_landmarks.part(n).y
                    mouth.append((x,y))
                    next_point = n+1
                    if n == 60:
                        next_point = 48
                    x2 = face_landmarks.part(next_point).x
                    y2 = face_landmarks.part(next_point).y
                    cv2.line(frame,(x,y),(x2,y2),(0,255,0),1)


                left_ear = calculate_EAR(leftEye)
                right_ear = calculate_EAR(rightEye)
                Mouth=calculate_mouth(mouth)

                EAR = (left_ear+right_ear)/2
                EAR = round(EAR,2)
                MEAR=round(Mouth,1)
                
                if EAR<0.26:
                    if MEAR>1:
                        m+=1
                    e+=1
                    print(EAR)
                    if e>40 and m>5:
                        cv2.putText(frame,"DROWSY",(20,100),cv2.FONT_HERSHEY_SIMPLEX,3,(0,0,255),4)
                        cv2.putText(frame,"Are you Sleepy?",(20,400),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)
                        print("Drowsy")
                        alarm=True
                        t = Thread(target=sound_alarm,args=["alarm.wav"])
                        t.deamon = True
                        t.start()
                else:
                    e=0
                    m=0
                    alarm=False
                cv2.imshow("Tracking", frame)
            else:
                break
            

            k = cv2.waitKey(1) & 0xff
            if k == 27 :
                break
    cv2.destroyWindow("Tracking")
    k = cv2.waitKey(1) & 0xff
    if k == 27 : break
cv2.destroyAllWindows()